# 13 图像配准

一张图片可能经过一个平移 / 旋转 / 拉伸变成另一张图像, 导致矩阵每个像素值差异很大, 不能直接比较. 

为了发现前后两张图片的相似性, 需要进行图像配准 (image registration).


### SSD

两张图片的表面上的距离可以简单地用 SSD (sum of square difference) 指标衡量:
$${\rm SSD}= \frac 1N\sum_{ij} (a_{ij} - b_{ij})^2$$

In [1]:
def SSD(img_A, img_B):
    """Sum of square difference between two images."""
    return ((img_A - img_B) ** 2).mean()

## 优化

实际上由于图像相差一个仿射变换, 需要先进行合适的仿射变换后再衡量. 假设两图为 $A,B$, 则需要找到仿射变换 $T$ 使
$$\min_T \ {\rm SSD}(A, T(B))$$

### 梯度下降

可以采用梯度下降方法来寻找这样的 $T$: 假设第 $k$ 步时得到 $T_k$, 则 $k+1$ 步时更新为 
$$T_{k+1} = T_k - \eta \nabla T_k$$

其中 $\eta$ 为学习率, 对矩阵 $(i,j)$ 元素, 梯度 $\nabla_{ij} T_k = \dfrac{\partial }{\partial T_{ij}}{\rm SSD}$ 可近似为 ($\Delta t\in \mathbb R$, $\Delta t\rightarrow 0$)
$$\nabla_{ij} T_k\approx  \frac{{\rm SSD}(A,(T+\Delta t E_{ij})(B)) - {\rm SSD}(A,(T-\Delta t E_{ij})(B))}{2\Delta t}$$

此处 $T+\Delta t E_{ij}$ 表示矩阵 $T$ 的 $(i,j)$ 元素增加 $\Delta t$.

### 变换分解

可以将仿射变换 $T$ 分解为旋转变换、伸缩变换、错切变换、平移变换的复合. 虽然参数更多, 但是每个参数的意义更明显.

In [2]:
import numpy as np
def registration(img_A, img_B, metric, step = 1e-2, lr = 1e-2, iters = 10):
    affine = np.eye(3) # initialization

    for iter in range(iters):
        for i in range(2): # only 2 x 3 entries in the affine matrix need training
            for j in range(3):
                # train (i,j) element of the affine matrix
                affine[i,j] += step
                grad = metric(img_A, img_B, affine)

                affine[i,j] -= step * 2
                grad -= metric(img_A, img_B, affine)

                affine[i,j] += step # restore
                grad /= 2 * step

                affine[i,j] -= lr * grad
    return affine


### 反向变换

欲计算图 $A,T(B)$ 的 ${\rm SSD}$ 指标, 只需要对每个点 $(x,y)$, 求出 $A(x,y)$ 与 $(T(B))(x,y)$ 的像素差值. 注意 $T(B)$ 在 $(x,y)$ 的像素值等于 $B$ 在 $T^{-1}(x,y)$ 处的像素值. 即

$${\rm SSD}(A,T(B)) = \frac 1N\sum_{x,y}\left(A(x,y) - T(B)(x,y)\right)^2
=\frac 1N\sum_{x,y}\left(A(x,y) - B(T^{-1}(x,y)\right)^2$$

假设 $T^{-1}(x,y) = (x',y')$, 也就是要求出每个 $\left(A(x,y) - B(x',y')\right)^2$.

### 图像插值

由于上述 $T^{-1}(x,y) = (x',y')$ 可能不是整点, 可以采用周围几个整点插值求得 $(x',y')$ 的像素值.

In [3]:
def affine_SSD(img_A, img_B, affine):
    """Compute SSD after applying an affine mapping on image B with image A."""
    # 1. construct the original homogenous coordinates
    h, w, d = img_B.shape[0], img_B.shape[1], img_B.size // (img_B.shape[0] * img_B.shape[1])
    coors = np.stack(np.meshgrid(np.arange(w), np.arange(h)))
    coors = np.transpose(coors, (1, 2, 0)).reshape((-1, 2))   # shape: (hw) x 2
    coors_hom = np.hstack((coors, np.ones((coors.shape[0], 1))))  # shape: (hw) x 3

    # 2. compute the inversed transformed coordinates
    coors2_hom = coors @ np.linalg.inv(affine)
    coors2 = coors2_hom[:,:-1] / coors2_hom[:,-1:]

    # 3. inquire the interpolated pixel
    coors2_grid = coors2.astype('int32')
    coors2_offs = coors2 - coors2_grid
    reference = lambda coor_x, coor_y: img_B[:, np.clip(coor_y, 0, h-1), np.clip(coor_x, 0, w-1)]
    img_B2 = np.prod(1 - coors2_offs, axis = 1) * reference(coors2_grid[:,0], coors2_grid[:,1])\
            + (1 - coors2_offs[:,0]) * coors2_offs[:,1] * reference(coors2_grid[:,0], coors2_grid[:,1] + 1)\
            + (1 - coors2_offs[:,1]) * coors2_offs[:,0] * reference(coors2_grid[:,1], coors2_grid[:,1] + 1)\
            + np.prod(coors2_offs, axis = 1) * reference(coors2_grid[:,0] + 1, coors2_grid[:,1] + 1)
    
    return ((img_A - img_B2) ** 2).mean()
